In [ ]:
#@title LICENSE

# Copyright 2024 DataStax, Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Use Vertex AI Extensions with a Custom Extension

## Overview


Vertex AI Extensions is a platform for creating and managing extensions that connect large language models to external systems via APIs. These external systems can provide LLMs with real-time data and perform data processing actions on their behalf. You can use pre-built or third-party extensions in Vertex AI Extensions.

Learn more about [Vertex AI Extensions](https://cloud.google.com/vertex-ai/docs/generative-ai/extensions/private/overview).

This notebook provides a simple getting started experience for the Vertex AI Extensions framework. This guide assumes that you are familiar with the Vertex AI Python SDK, [LangChain](https://python.langchain.com/docs/get_started/introduction), [OpenAPI specification](https://swagger.io/specification/), and [Cloud Run](https://cloud.google.com/run/docs).

### Objective

In this tutorial, you learn how to create an extension service backend on Cloud Run, register the extension with Vertex, and then use the extension in an application.

The steps performed include:

- Creating a simple service running on Cloud Run
- Creating an OpenAPI 3.1 YAML file for the Cloud Run service
- Registering the service as an extension with Vertex AI
- Using the extension to respond to user queries
- Integrate LangChain into the reasoning for an extension

### Additional Information

This tutorial uses the following Google Cloud services and resources:

- Vertex AI Extensions
- Cloud Run

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.11

### Authenticate your Google Cloud account

You must authenticate to Google Cloud to begin.

In [ ]:
import sys

if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth
    auth.authenticate_user()

### Installation

This tutorial the latest Python SDK for Vertex AI.

Run the following command to download the SDK:

In [ ]:
%pip install -U "google-cloud-aiplatform" \
"openapi-schema-pydantic==1.2.4" \
"openapi-pydantic==0.3.2" \
"google-cloud-storage" \
"shapely<2"

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.
1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).
1. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).
1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).
1. Your project must also be allowlisted for the Vertex AI Extension Private Preview.
1. This notebook requires that you have the following permissions for your GCP project:
- `roles/aiplatform.user`

### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[PROJECT_ID]"  # @param {type:"string"}

# Set the project id
!gcloud config set project {PROJECT_ID}

### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_NAME = "[BUCKET_NAME]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
!gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### Import libraries



In [ ]:
import vertexai

from google.cloud.aiplatform import llm_extension

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
vertexai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

## Creating an API backend service

In this tutorial, you will create and deploy the Astra DB CRUD Extension to Google Cloud Run, with authentication. Note that the backend service can be deployed outside of Cloud Run as well, but we will assume that you're using Cloud Run in this tutorial

### Deploy the API service to Cloud Run

First, we define the path of the extension for future use

In [ ]:
EXTENSION_PATH = "../astra-crud-extension"

Next, you deploy the service to Cloud Run. However, you might need to log in once more to deploy.

In [ ]:
!gcloud auth login

In [ ]:
!gcloud run deploy astra-crud-extension --region=us-central1 --allow-unauthenticated --source ../astra-crud-extension --no-user-output-enabled

List the most recent Cloud Run service that was deployed, then you'll copy its URL to the next cell:

In [ ]:
!gcloud run services list

In [ ]:
# @title Copy paste the output from the previous command here
service_url = "[SERVICE_URL]"  # @param {type:"string"}

### Create an OpenAPI spec

Your Vertex Extension requires an OpenAPI 3.1 YAML file that defines routes, URL, HTTP methods, requests, and responses from your "backend" service. The following code creates a YAML file that you need to upload to your Cloud Storage bucket.

In [ ]:
openapi_yaml = f"""
openapi: 3.1.0
info:
  title: Astra Vertex Extension
  description: An extension to perform CRUD actions on data within your Astra Database.
  version: 1.0.0
servers:
  - url: '{service_url}'
paths:
  /health:
    get:
      operationId: health
      summary: A simple health check endpoint
      responses:
        '200':
          description: Successful response
          content:
            application/json:
              schema:
                type: object
                properties:
                  message:
                    type: string
  /readData:
    post:
      operationId: readData
      summary: Search for data within the database using a filter
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                filter:
                  type: object
                  description: Key-value pairs to filter the data
      responses:
        '200':
          description: Successful response
          content:
            application/json:
              schema:
                type: array
                items:
                  type: object
  /updateData:
    post:
      operationId: updateData
      summary: Update existing data within the database
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                filter:
                  type: object
                fieldUpdate:
                  type: object
      responses:
        '200':
          description: Data updated successfully
          content:
            application/json:
              schema:
                type: array
                items:
                  type: object
  /insertData:
    post:
      operationId: insertData
      summary: Insert new data into the database
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                data:
                  type: array
                  items:
                    type: object
      responses:
        '200':
          description: Data inserted successfully
          content:
            application/json:
              schema:
                type: array
                items:
                  type: object
  /deleteData:
    post:
      operationId: deleteData
      summary: Delete existing data within the database
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                filter:
                  type: object
      responses:
        '200':
          description: Data deleted successfully
          content:
            application/json:
              schema:
                type: array
                items:
                  type: object
"""

print(openapi_yaml)

In [ ]:
%store openapi_yaml >../astra-crud-extension-api/extension.yaml

Upload the OpenAPI YAML to your Cloud Storage bucket.

In [ ]:
from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)
blob_name = f"{EXTENSION_PATH}/extension.yaml"
blob = bucket.blob(blob_name)
blob.upload_from_filename(f"{EXTENSION_PATH}-api/extension.yaml")

### Test the service locally

First, check that your service can accept simple HTTP `GET` requests. We will use the `moviereviews` table that gets created when loading sample data into an Astra DB instance. 

First, set your DATASTAX_VERTEX_AI_TOKEN as the concatenation of your ASTRA_DB_APPLICATION_TOKEN, ASTRA_DB_API_ENDPOINT, and ASTRA_DB_TABLE (see the README for more details)

In [ ]:
DATASTAX_VERTEX_AI_TOKEN = "AstraCS:<...>;https://<...>.apps.astra.datastax.com;moviereviews"  # @param {type: "string"}

In [ ]:
import requests

r = requests.post(service_url + "/readData", json={}, headers = {"token": DATASTAX_VERTEX_AI_TOKEN})

print(f"Status Code: {r.status_code}, Content: {r.text}")

## Creating and using a custom extension

### Create the extension

Now that you've set up the service to fulfill extension requests, you can create the extension itself.

In [ ]:
SECRET_ID = "DATASTAX_VERTEX_AI_TOKEN"

extension_astra = llm_extension.Extension.create(
    display_name = "Perform a CRUD Operation on Astra DB",
    description = "Inserts, loads, updates, or deletes data from Astra DB and returns it to the user",
    manifest = {
        "name": "astra_tool",
        "description": "Access and process data from AstraDB",
        "api_spec": {
            "open_api_gcs_uri": f"gs://{BUCKET_NAME}/{EXTENSION_PATH}/extension.yaml"
        },
        "authConfig": {
            "authType": "API_KEY_AUTH",
            "apiKeyConfig": {
                "name": "token",
                "apiKeySecret": f"projects/{PROJECT_ID}/secrets/{SECRET_ID}/versions/latest",
                "httpElementLocation": "HTTP_IN_HEADER",
            },
        }
    },
)

extension_astra

Now that you've create your extension, let's confirm that it's registered:

In [ ]:
print("Name:", extension_astra.gca_resource.name)
print("Display Name:", extension_astra.display_name)
print("Description:", extension_astra.gca_resource.description)

print(extension_astra.to_dict())

And you can test the functionality of the extension by executing it:

In [ ]:
extension_astra.execute("health", operation_params={})

In [ ]:
extension_astra.execute("readData",
    operation_params = {},
)